In [4]:
# ✅ Importar librerías necesarias
import tensorflow as tf
from tensorflow.keras.models import load_model, Model, Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input, Flatten
from tensorflow.keras.optimizers import Adam
import pandas as pd
import numpy as np

# ✅ Rutas de los archivos
ruta_test_csv = r'C:\Users\mikel\OneDrive\Documentos\TB-DS-BIO-23.09.24\REPOSITORIOS\Mikel\clip_count\test.csv'
ruta_test_img = r'C:\Users\mikel\OneDrive\Documentos\TB-DS-BIO-23.09.24\REPOSITORIOS\Mikel\clip_count\test'
ruta_modelo = r'C:\Users\mikel\OneDrive\Documentos\TB-DS-BIO-23.09.24\REPOSITORIOS\Mikel\dsft2409_mikeltelo\PROJECTS\kaggle_clip_counting\modelo_final\modelo_clip_count_p.h5'
ruta_resultados = r'C:\Users\mikel\OneDrive\Documentos\TB-DS-BIO-23.09.24\REPOSITORIOS\Mikel\dsft2409_mikeltelo\PROJECTS\kaggle_clip_counting\submission_p_final.csv'

# ✅ Cargar el modelo base
modelo_base = load_model(ruta_modelo)
print(f"✅ Modelo base cargado correctamente desde '{ruta_modelo}'")

# ✅ Inspeccionar si el modelo es Sequential
if isinstance(modelo_base, Sequential):
    print("🔄 El modelo base es Sequential. Añadiendo entrada explícita.")
    entrada = Input(shape=(128, 128, 1))  # Ajusta la forma si es diferente
    x = modelo_base(entrada, training=False)  # Pasamos la entrada explícita
else:
    print("🔄 El modelo base es Functional.")
    entrada = modelo_base.input
    x = modelo_base.output

# ✅ Congelar capas base
for capa in modelo_base.layers:
    capa.trainable = False

# ✅ Añadir capas personalizadas
if len(x.shape) == 4:
    print("🔄 Salida con 4 dimensiones, aplicando GlobalAveragePooling2D.")
    x = GlobalAveragePooling2D()(x)
elif len(x.shape) == 2:
    print("🔄 Salida con 2 dimensiones, aplicando capa Dense directamente.")
    x = Dense(128, activation='relu')(x)
else:
    raise ValueError("❌ Dimensiones de salida inesperadas. Revisa el modelo base.")

# ✅ Capas adicionales
x = Dense(128, activation='relu')(x)
x = Dense(64, activation='relu')(x)
salida = Dense(1, activation='linear')(x)  # Salida para regresión

# ✅ Crear nuevo modelo funcional
modelo_transfer = Model(inputs=entrada, outputs=salida)

# ✅ Compilar el modelo
modelo_transfer.compile(optimizer=Adam(learning_rate=0.001),
                        loss='mean_squared_error',
                        metrics=['mean_absolute_error'])

# ✅ Mostrar resumen
print("✅ Modelo ajustado para Transfer Learning listo para ser entrenado.")
modelo_transfer.summary()



✅ Modelo base cargado correctamente desde 'C:\Users\mikel\OneDrive\Documentos\TB-DS-BIO-23.09.24\REPOSITORIOS\Mikel\dsft2409_mikeltelo\PROJECTS\kaggle_clip_counting\modelo_final\modelo_clip_count_p.h5'
🔄 El modelo base es Sequential. Añadiendo entrada explícita.
🔄 Salida con 2 dimensiones, aplicando capa Dense directamente.
✅ Modelo ajustado para Transfer Learning listo para ser entrenado.


Model: "functional_52"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 128, 128, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_2 (Sequential)       │ (None, 1)              │     3,305,089 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,330,178 (12.70 MB)

 Trainable params: 25,089 (98.00 KB)

 Non-trainable params: 3,305,089 (12.61 MB)

In [5]:
# ✅ Cargar datos de prueba
df_test = pd.read_csv(ruta_test_csv)

# ✅ Función de preprocesamiento para imágenes de prueba
IMG_SIZE = (128, 128)
BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE

def cargar_datos_test(df, ruta_img):
    ids = df['id'].values
    
    def preprocesar_imagen(id):
        img_path = tf.strings.join([ruta_img, "/clips-", tf.strings.as_string(id), ".png"])
        img = tf.io.read_file(img_path)
        img = tf.image.decode_png(img, channels=1)
        img = tf.image.resize(img, IMG_SIZE)
        img = img / 255.0
        return img
    
    dataset = tf.data.Dataset.from_tensor_slices(ids)
    dataset = dataset.map(lambda id: preprocesar_imagen(id), num_parallel_calls=AUTOTUNE)
    dataset = dataset.batch(BATCH_SIZE).prefetch(AUTOTUNE)
    return dataset

# ✅ Crear Dataset de Prueba
dataset_test = cargar_datos_test(df_test, ruta_test_img)
print("✅ Datos de prueba preparados correctamente.")


✅ Datos de prueba preparados correctamente.


In [8]:
# ✅ Importar librerías necesarias
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# ✅ Parámetros globales
IMG_SIZE = (128, 128)
BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE

# ✅ Rutas de los archivos
ruta_train_csv = r'C:\Users\mikel\OneDrive\Documentos\TB-DS-BIO-23.09.24\REPOSITORIOS\Mikel\clip_count\train.csv'
ruta_test_img = r'C:\Users\mikel\OneDrive\Documentos\TB-DS-BIO-23.09.24\REPOSITORIOS\Mikel\clip_count\test'

# ✅ Cargar datos
df = pd.read_csv(ruta_train_csv)

# ✅ Dividir en entrenamiento (80%) y validación (20%)
df_train, df_val = train_test_split(df, test_size=0.2, random_state=42)

# ✅ Función de preprocesamiento para imágenes
def cargar_datos(df, ruta_img):
    ids = df['id'].values
    etiquetas = df['clip_count'].values  # Ajusta si la columna tiene otro nombre

    def preprocesar_imagen(id, etiqueta):
        img_path = tf.strings.join([ruta_img, "/clips-", tf.strings.as_string(id), ".png"])
        img = tf.io.read_file(img_path)
        try:
            img = tf.image.decode_png(img, channels=1)
        except tf.errors.InvalidArgumentError:
            print(f"❌ Error al decodificar la imagen {img_path}")
            img = tf.zeros((IMG_SIZE[0], IMG_SIZE[1], 1))
        img = tf.image.resize(img, IMG_SIZE)
        img = img / 255.0
        return img, etiqueta
    
    dataset = tf.data.Dataset.from_tensor_slices((ids, etiquetas))
    dataset = dataset.map(lambda id, etiqueta: preprocesar_imagen(id, etiqueta), num_parallel_calls=AUTOTUNE)
    dataset = dataset.batch(BATCH_SIZE).prefetch(AUTOTUNE)
    return dataset

# ✅ Crear Dataset de Entrenamiento y Validación
dataset_train = cargar_datos(df_train, ruta_test_img)
dataset_val = cargar_datos(df_val, ruta_test_img)

print("✅ Datasets de entrenamiento y validación preparados correctamente.")



✅ Datasets de entrenamiento y validación preparados correctamente.


In [9]:
# ✅ Generar predicciones
predicciones = modelo_transfer.predict(dataset_test).flatten()
df_test['clip_count'] = np.round(predicciones).astype(int)
print("✅ Predicciones generadas correctamente.")


157/157 ━━━━━━━━━━━━━━━━━━━━ 10s 60ms/step
✅ Predicciones generadas correctamente.


In [10]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# ✅ Comprobar si existen etiquetas reales en el dataset de prueba
if 'clip_count' in df_test.columns:
    etiquetas_reales = df_test['clip_count'].values
    etiquetas_predichas = np.round(predicciones).astype(int)
    
    # ✅ Calcular Métricas
    mae = mean_absolute_error(etiquetas_reales, etiquetas_predichas)
    mse = mean_squared_error(etiquetas_reales, etiquetas_predichas)
    rmse = np.sqrt(mse)
    r2 = r2_score(etiquetas_reales, etiquetas_predichas)
    
    # ✅ Mostrar Métricas
    print("📊 **Métricas del Modelo en el Conjunto de Prueba:**")
    print(f"✅ MAE (Error Absoluto Medio): {mae:.2f}")
    print(f"✅ MSE (Error Cuadrático Medio): {mse:.2f}")
    print(f"✅ RMSE (Raíz del Error Cuadrático Medio): {rmse:.2f}")
    print(f"✅ R2 Score (Coeficiente de Determinación): {r2:.2f}")
else:
    print("❌ El conjunto de prueba no contiene etiquetas reales ('clip_count'). No se pueden calcular métricas.")


📊 **Métricas del Modelo en el Conjunto de Prueba:**
✅ MAE (Error Absoluto Medio): 0.00
✅ MSE (Error Cuadrático Medio): 0.00
✅ RMSE (Raíz del Error Cuadrático Medio): 0.00
✅ R2 Score (Coeficiente de Determinación): 1.00


In [11]:
# ✅ Crear un DataFrame con las Predicciones
df_resultados = pd.DataFrame({
    'id': df_test['id'],  # Usamos los IDs del archivo original
    'clip_count': np.round(predicciones).astype(int)  # Redondeamos las predicciones
})

# ✅ Guardar en un Archivo CSV
ruta_salida = r'C:\Users\mikel\OneDrive\Documentos\TB-DS-BIO-23.09.24\REPOSITORIOS\Mikel\dsft2409_mikeltelo\PROJECTS\kaggle_clip_counting\submission_p_final_tl.csv'
df_resultados.to_csv(ruta_salida, index=False)

print(f"✅ Archivo de salida guardado en '{ruta_salida}'")
print("📊 **Vista previa del archivo de salida:**")
print(df_resultados.head())


✅ Archivo de salida guardado en 'C:\Users\mikel\OneDrive\Documentos\TB-DS-BIO-23.09.24\REPOSITORIOS\Mikel\dsft2409_mikeltelo\PROJECTS\kaggle_clip_counting\submission_p_final_tl.csv'
📊 **Vista previa del archivo de salida:**
      id  clip_count
0  45001           5
1  45002           3
2  45003           3
3  45004           3
4  45005           1
